In [129]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.callbacks import ModelCheckpoint

In [130]:
data=pd.read_csv('airline_sentiment_analysis.csv')
data=data[['airline_sentiment','text']]
data.head()

,airline_sentiment,text
0,positive,@VirginAmerica plus you've added commercials t...
1,negative,@VirginAmerica it's really aggressive to blast...
2,negative,@VirginAmerica and it's a really big bad thing...
3,negative,@VirginAmerica seriously would pay $30 a fligh...
4,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [131]:
data=data[['airline_sentiment','text']]

In [132]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11541 entries, 0 to 11540
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  11541 non-null  object
 1   text               11541 non-null  object
dtypes: object(2)
memory usage: 180.5+ KB


In [133]:
data['text'][0]

"@VirginAmerica plus you've added commercials to the experience... tacky."

In [134]:
Y = pd.get_dummies(data['airline_sentiment'],drop_first=True).values

In [135]:
Y

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=uint8)

In [136]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [137]:
data.iloc[1]

airline_sentiment                                             negative
text                 virginamerica its really aggressive to blast o...
Name: 1, dtype: object

In [138]:
data['text']= [x.split(maxsplit=1)[1] for x in data['text']]


In [139]:
type(data['text'])

pandas.core.series.Series

In [141]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [142]:
X = tokenizer.texts_to_sequences(data['text'].values)


In [143]:
X = pad_sequences(X)
print(X[75])

[   0    0    0    0    0    0    0    0    0    0    0    0   15 1383
   11  285  205   29   26   16   17  493    1   62  237   10  176  321
   53   15   17]


In [196]:
X.shape[1]

31

In [145]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 31, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 31, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 197       
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


In [146]:
X_train, x_test, Y_train, y_test = train_test_split(X,Y, test_size = 0.34, random_state = 42)
X_valid, X_test , Y_valid, Y_test = train_test_split(x_test,y_test, test_size = 0.5, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)
print(X_test.shape,Y_test.shape)

(7617, 31) (7617, 1)
(1962, 31) (1962, 1)
(1962, 31) (1962, 1)


In [147]:
checkpt= ModelCheckpoint('model.h5',monitor='val_accuracy', save_best_only=True, mode='max',verbose=1)

In [148]:
batch_size = 32
history= model.fit(X_train, Y_train,validation_data=(X_valid,Y_valid), epochs = 20, batch_size=batch_size, verbose = 1,callbacks=[checkpt])

Epoch 1/20
239/239 [==============================] - ETA: 0s - loss: 0.3485 - accuracy: 0.8582
Epoch 00001: val_accuracy improved from -inf to 0.90826, saving model to model.h5
239/239 [==============================] - 17s 71ms/step - loss: 0.3485 - accuracy: 0.8582 - val_loss: 0.2210 - val_accuracy: 0.9083
Epoch 2/20
239/239 [==============================] - ETA: 0s - loss: 0.1978 - accuracy: 0.9225
Epoch 00002: val_accuracy improved from 0.90826 to 0.91947, saving model to model.h5
239/239 [==============================] - 16s 68ms/step - loss: 0.1978 - accuracy: 0.9225 - val_loss: 0.1948 - val_accuracy: 0.9195
Epoch 3/20
238/239 [============================>.] - ETA: 0s - loss: 0.1488 - accuracy: 0.9435 ETA: 1s -
Epoch 00003: val_accuracy improved from 0.91947 to 0.91998, saving model to model.h5
239/239 [==============================] - 16s 65ms/step - loss: 0.1488 - accuracy: 0.9435 - val_loss: 0.2044 - val_accuracy: 0.9200
Epoch 4/20
238/239 [============================>.]

In [149]:
model.evaluate(X_test, Y_test, verbose = 1, batch_size = batch_size)

62/62 [==============================] - 0s 8ms/step - loss: 0.6003 - accuracy: 0.9052


[0.6002779006958008, 0.9051987528800964]

In [150]:
twt = ['Such a good service, comfortable']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=31, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0][0]
if sentiment<0.5:
    print("negative")
else:
    print("positive")

1/1 - 0s
positive


In [151]:
twt = ['Worst Service']
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=31, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0][0]
if sentiment<0.5:
    print("negative")
else:
    print("positive")

1/1 - 0s
negative


In [79]:
!pip install pyramid

In [80]:
from wsgiref.simple_server import make_server
from pyramid.config import Configurator
from pyramid.response import Response

def hello_world(request):
    return Response('Hello World!')

if __name__ == '__main__':
    with Configurator() as config:
        config.add_route('hello', '/')
        config.add_view(hello_world, route_name='hello')
        app = config.make_wsgi_app()
    server = make_server('0.0.0.0', 6543, app)
    server.serve_forever()


KeyboardInterrupt: 

In [81]:
! pip install -r requirements.txt

In [177]:
from keras.models import load_model
model=load_model('model.h5')

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    

In [178]:
twt = ['Such bad service']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=31, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0][0]
print(sentiment)
if sentiment<0.5:
    print("negative")
else:
    print("positive")

1/1 - 0s
0.020496726
negative


In [194]:
def Predict(text):
    text=text.lower()
    text=re.sub('[^a-zA-z0-9\s]','',text)
    text = tokenizer.texts_to_sequences([text])
    text = pad_sequences(text, maxlen=31, dtype='int32', value=0)
    sentiment = model.predict(text,batch_size=1,verbose = 0)[0][0]
    print(sentiment)
    if sentiment<0.5:
        return("negative")
    else:
        return("positive")

In [195]:
Predict('Such bad service')

0.020496726


'negative'

In [193]:
a='Such bad service ,@food'
a=a.lower()
a=re.sub('[^a-zA-z0-9\s]','',a)
a

'such bad service food'

In [158]:
from wsgiref.simple_server import make_server
from pyramid.config import Configurator
from pyramid.response import Response

def hello_world(request):
    return Response('Hello World')

if __name__ == '__main__':
    with Configurator() as config:
        config.add_route('hello', '/')
        config.add_view(hello_world, route_name='hello')
        app = config.make_wsgi_app()
    server = make_server('127.0.0.1', 5000, app)
    server.serve_forever()


KeyboardInterrupt: 

In [128]:
#!pip install waitress
from waitress import serve
from pyramid.config import Configurator
from pyramid.response import Response

name='Soham'

def hello_world(request):
    print('Incoming request')
    return Response('<!DOCTYPE html><html lang="en"><head>    <title>Quick Tutorial: {{ name }}</title></head><body><h1>Hi {}</h1></body></html>'.format(name))


if __name__ == '__main__':
    with Configurator() as config:
        config.add_route('hello', '/')
        config.add_view(hello_world, route_name='hello')
        app = config.make_wsgi_app()
    serve(app, host='127.0.0.1', port=6543)

Serving on http://DESKTOP-6PBPHLE:6543
Incoming request
Incoming request
Incoming request
